## Generate Prompt Template File from Mol File
Generate potential prompt templates from a molecule given a sample molecule

In [4]:
from mofa.model import LigandTemplate
from dataclasses import asdict
from rdkit import Chem
import itertools
import yaml

Configuration

In [5]:
mol_file = 'example-mols/cyano-with-benzenes.mol'  # Molecule forming the template
anchor_type = 'cyano'
anchor_group = 'C#N' #  SMARTS of group which will attach to metal centers, "anchors". First atom should be the carbon from the anchor which connects to the rest of the molecule - "link carbon"
dummy_element = 'Fr'
prompt_size: int = 5  # Number of bonds between the link carbon and the end of a prompt group

Load the molecule 

In [6]:
mol = Chem.MolFromMolFile(mol_file, removeHs=False)

Find the rings in the molecule. We'll use them when making sure the prompts are correct

In [7]:
all_rings = list(map(set, mol.GetRingInfo().AtomRings()))
all_rings

[{2, 3, 4, 5, 20, 21}, {12, 13, 14, 15, 18, 19}]

The groups which connect to the rest of the molecule

In [8]:
anchor_matches = mol.GetSubstructMatches(Chem.MolFromSmarts('C#N'))
assert len(anchor_matches) == 2

For each, find the atoms which are suitable for prompts (close enough, not part of ring)

In [9]:
prompt_xyzs = []
for aid, anchor in enumerate(anchor_matches):
    link_carbon = anchor[0]

    # Get all bonds within the radius
    bonds = list(Chem.FindAtomEnvironmentOfRadiusN(mol, prompt_size, useHs=True, rootedAtAtom=link_carbon))

    # Get the atoms which are part of the group
    prompt_atoms = set()
    for bond_id in bonds:
        bond = mol.GetBondWithIdx(bond_id)
        prompt_atoms.add(bond.GetBeginAtomIdx())
        prompt_atoms.add(bond.GetEndAtomIdx())
    prompt_atoms = sorted(prompt_atoms)
    print(f'Anchor {aid} prompt includes {len(prompt_atoms)} atoms: {", ".join(map(str, prompt_atoms))}')

    # Make sure that if we have one atom from a ring, we include all of the ring
    for ring in all_rings:
        if len(ring.intersection(prompt_atoms)) > 0:
            assert len(ring.difference(prompt_atoms)) == 0, 'Included some but not all atoms in a ring'

    # Detect which atom connects to the rest of the molecule
    is_outside = []
    for atom_id in prompt_atoms:
        atom = mol.GetAtomWithIdx(atom_id)
        neighbors = [n.GetIdx() for n in atom.GetNeighbors()]
        is_outside.append(any(n not in prompt_atoms for n in neighbors))
    assert is_outside.count(True) == 1, 'More than one atom is at the edge of the prompt'
    outside_atom = prompt_atoms[is_outside.index(True)]
    print(f'Atom {outside_atom} connects with the rest of the molecule')

    # Save the prompt group such that atom which connects to the rest of the molecule is the first atom
    xyz = f'{len(prompt_atoms)}\nPrompt {aid}\n'
    not_special = sorted(set(prompt_atoms).difference(anchor).difference([outside_atom]))
    conf = mol.GetConformer()
    for atom_id in [outside_atom] + not_special + list(anchor):
        a = mol.GetAtomWithIdx(atom_id)
        s = a.GetSymbol()
        c = conf.GetAtomPosition(atom_id)
        xyz += f"{s} {c[0]} {c[1]} {c[2]}\n"
    prompt_xyzs.append(xyz)

    with open(f'prompt-{aid}.xyz', 'w') as fp:
        print(xyz, file=fp)

Anchor 0 prompt includes 13 atoms: 0, 1, 2, 3, 4, 5, 6, 20, 21, 22, 23, 34, 35
Atom 6 connects with the rest of the molecule
Anchor 1 prompt includes 13 atoms: 11, 12, 13, 14, 15, 16, 17, 18, 19, 30, 31, 32, 33
Atom 11 connects with the rest of the molecule


Make then save the ligand template

In [10]:
template = LigandTemplate(
    anchor_type=anchor_type,
    xyzs=prompt_xyzs,
    dummy_element=dummy_element
)

In [11]:
with open(f'template_{anchor_type}_size={prompt_size}.yml', 'w') as fp:
    yaml.safe_dump(asdict(template), fp)